In [7]:
# this notebook demonstrates how to use the Open-Meteo API to get historical weather data

# based on:
# https://open-meteo.com/en/docs/historical-weather-api

In [8]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


In [9]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [10]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    # geographical coordinates for states: https://www.latlong.net/category/states-236-14.html
    # for TX assumed:
	# "latitude": 31.000000,
	# "longitude": -100.000000,
    # for WI assumed:
	# "latitude": 44.500000,
	# "longitude": -89.500000,
    # for CA assumed: 
	"latitude": 36.778259,
	"longitude": -119.417931,
	"start_date": "2011-01-29",
	"end_date": "2016-06-19",
	"daily": ["weather_code", "rain_sum", "sunshine_duration", "temperature_2m_max", "temperature_2m_min"]   
    # add more items to list if needed, from docs: https://open-meteo.com/en/docs/historical-weather-api -> Daily Parameter Definition
}
responses = openmeteo.weather_api(url, params=params)


In [11]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 36.8014030456543°N -119.44808959960938°E
Elevation 144.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [12]:
daily = response.Daily()
weathers_code = daily.Variables(0).ValuesAsNumpy()
rain_sum = daily.Variables(1).ValuesAsNumpy()
sunshine_duration = daily.Variables(2).ValuesAsNumpy()
temperature_2m_max = daily.Variables(3).ValuesAsNumpy()
temperature_2m_min = daily.Variables(4).ValuesAsNumpy()

In [13]:

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weathers_code"] = weathers_code
daily_data["rain_sum"] = rain_sum
daily_data["sunshine_duration [sec]"] = sunshine_duration
daily_data["temperature_2m_max"] = temperature_2m_max
daily_data["temperature_2m_min"] = temperature_2m_min


In [14]:
daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)


                          date  weathers_code  rain_sum  sunshine_duration  \
0    2011-01-29 00:00:00+00:00            2.0       0.0       33701.898438   
1    2011-01-30 00:00:00+00:00           55.0       8.2       23953.308594   
2    2011-01-31 00:00:00+00:00           53.0       1.2       31584.457031   
3    2011-02-01 00:00:00+00:00            3.0       0.0       34066.929688   
4    2011-02-02 00:00:00+00:00            1.0       0.0       34191.574219   
...                        ...            ...       ...                ...   
1964 2016-06-15 00:00:00+00:00            0.0       0.0       49398.054688   
1965 2016-06-16 00:00:00+00:00            0.0       0.0       49413.078125   
1966 2016-06-17 00:00:00+00:00            0.0       0.0       49425.312500   
1967 2016-06-18 00:00:00+00:00            0.0       0.0       49434.734375   
1968 2016-06-19 00:00:00+00:00            0.0       0.0       49441.312500   

      temperature_2m_max  temperature_2m_min  
0              1

In [15]:
daily_dataframe[daily_dataframe["weathers_code"] == 51.0]

,date,weathers_code,rain_sum,sunshine_duration,temperature_2m_max,temperature_2m_min
17,2011-02-15 00:00:00+00:00,51.0,1.1,33080.320312,16.811001,7.861000
52,2011-03-22 00:00:00+00:00,51.0,1.2,37945.710938,13.811001,1.911000
74,2011-04-13 00:00:00+00:00,51.0,0.2,43200.000000,19.011000,7.161000
82,2011-04-21 00:00:00+00:00,51.0,1.6,41143.070312,21.661001,11.711000
83,2011-04-22 00:00:00+00:00,51.0,0.7,41661.742188,19.961000,9.011001
...,...,...,...,...,...,...
1916,2016-04-28 00:00:00+00:00,51.0,0.5,41146.343750,18.811001,9.011001
1920,2016-05-02 00:00:00+00:00,51.0,0.2,45728.683594,28.561001,12.711000
1923,2016-05-05 00:00:00+00:00,51.0,0.2,44304.652344,26.861000,13.511001
1932,2016-05-14 00:00:00+00:00,51.0,0.1,47571.339844,32.310997,15.011001


In [16]:
daily_dataframe[daily_dataframe["rain_sum"] > 0.0]

,date,weathers_code,rain_sum,sunshine_duration,temperature_2m_max,temperature_2m_min
1,2011-01-30 00:00:00+00:00,55.0,8.200000,23953.308594,13.811001,6.611000
2,2011-01-31 00:00:00+00:00,53.0,1.200000,31584.457031,10.161000,5.361000
17,2011-02-15 00:00:00+00:00,51.0,1.100000,33080.320312,16.811001,7.861000
18,2011-02-16 00:00:00+00:00,63.0,18.299999,26465.978516,16.361000,8.111000
19,2011-02-17 00:00:00+00:00,53.0,2.300000,33437.027344,12.461000,1.661000
...,...,...,...,...,...,...
1920,2016-05-02 00:00:00+00:00,51.0,0.200000,45728.683594,28.561001,12.711000
1923,2016-05-05 00:00:00+00:00,51.0,0.200000,44304.652344,26.861000,13.511001
1924,2016-05-06 00:00:00+00:00,55.0,6.400001,35948.203125,19.061001,12.511001
1932,2016-05-14 00:00:00+00:00,51.0,0.100000,47571.339844,32.310997,15.011001


In [17]:
daily_dataframe["rain_sum"].value_counts()

rain_sum
0.000000     1656
0.300000       18
0.200000       16
0.100000       16
0.600000        9
             ... 
7.300000        1
7.800000        1
2.200000        1
23.800001       1
2.300000        1
Name: count, Length: 187, dtype: int64